In [1]:
import os
from langchain_openai import ChatOpenAI
from langgraph.graph import Graph
from langgraph.prebuilt import create_react_agent
import gradio as gr
from datetime import datetime

# Import tools
from tools.calculator import CalculatorTool
from tools.city_recommender import CityRecommendationTool
from tools.hotel_search import HotelSearchTool
from tools.flight_search import FlightSearchTool
from tools.attractions_tool import CityAttractionsTool
from city_vector_db import CityVectorDB

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def initialize_tools():

    vector_db = CityVectorDB()
    vector_db.load("cities_vector_db")

    tools = [
        FlightSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        CityAttractionsTool(),
        HotelSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        CityRecommendationTool(vector_db=vector_db),
        CalculatorTool()
    ]
    return tools

In [3]:
tools = initialize_tools()

In [4]:
system_prompt = f"""You are a helpful travel assistant named Fred.

Follow these steps in order:

1. CITY RECOMMENDATION:
   - Use the city_recommendation_tool and the user's given preference EXACTLY ONCE to suggest a city
   
2. FLIGHT SEARCH:
   - Use the user's given source as the departure_airport and the first recommended city as the arrival_airport
   - Ask for specific travel dates (departure and return, if return is not given, ask for it and get the duration of the trip)
   - Use flight_search_tool with IATA codes
   - Format: {{
       'departure_airport': '[IATA code]',
       'arrival_airport': '[IATA code]',
       'departure_date': 'YYYY-MM-DD',
       'return_date': 'YYYY-MM-DD'  # Optional for one-way flights
   }}

3. HOTEL SEARCH
   - Look for hotels in the recommended city for the dates of the flight (departure and return date) and pick the cheapest one

4. ATTRACTIONS:
   - Once the hotel and flight are selected, use the city_attractions_tool to get the top attractions in the recommended city. Make sure you match the correct formats for the tool 

6. END:
   - Once the user is done with the conversation, thank them and end the conversation
   
Remember that today's date is {datetime.now().strftime('%Y-%m-%d')}. Only search for future dates.
Do not repeat city recommendations unless explicitly asked by the user.

For eg. If the user's name is John Lincoln, they like skiing and are flying from New York. First, recommend a city using the city_recommendation_tool. Ask the user where they are flying from and the dates of travel if they haven't given it. Use the flight_search_tool to get the flight details.
Then look for hotels in the destination for the dates of the flight (departure and return date) using the hotel_search_tool. Then look for attractions in the destination using the city_attractions_tool. Inform the user of their whole itinerary."""

In [5]:
def create_agent(system_prompt):
    model = ChatOpenAI(model="gpt-4-turbo")
    tools = initialize_tools()
    return create_react_agent(model, tools=tools, state_modifier=system_prompt, debug=True)

In [6]:
agent = create_agent(system_prompt)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
from langchain.callbacks import get_openai_callback

messages = [("user", "my name is sam controlman and i want to plan a holiday from 25th to 30th december. I like flying and am flying from ABCD.")]

with get_openai_callback() as cb:
    response = agent.invoke({"messages": messages})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost:.4f}")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user',
               'my name is sam controlman and i want to plan a holiday from '
               '25th to 30th december. I like flying and am flying from '
               'ABCD.')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user',
  'my name is sam controlman and i want to plan a holiday from 25th to 30th '
  'december. I like flying and am flying from ABCD.')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='my name is sam controlman and i want to plan a holiday from 25th to 30th december. I like flying and am flying from ABCD.', additional_kwargs={}, response_metadata={}, id='ef038b95-a1ed-419c-96dc-d2b91c328d49')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='my name is sam controlman and i want to plan a holiday from 25th to 30th 

In [8]:
print(response['messages'][-1].content)

Queenstown, New Zealand, is an excellent choice for your holiday as it's known for its adventure activities and stunning natural beauty. It's perfect for someone who enjoys flying, with activities like skydiving and jet boating available.

Let's proceed with booking your flight. Since you're flying from ABCD, please confirm if this is a correct airport code, or provide the nearest major airport IATA code.
